In [112]:
import os
os.getcwd()


'C:\\Users\\banda\\Documents\\linux\\neuralnets\\Natural Language  Processing with Classification and Vector Spaces\\week2'

In [130]:
from utils import process_tweet
import pdb
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pandas as pd
import nltk
import string
from nltk.tokenize import TweetTokenizer
from os import getcwd

In [114]:
filepath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filepath)

In [131]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

train_y = np.append(np.ones(len(train_pos), dtype=np.int8), np.zeros(len(train_neg), dtype=np.int8))
test_y = np.append(np.ones(len(test_pos), dtype=np.int8), np.zeros(len(test_neg), dtype=np.int8))
train_y, test_y

(array([1, 1, 1, ..., 0, 0, 0], dtype=int8),
 array([1, 1, 1, ..., 0, 0, 0], dtype=int8))

### Process Data

In [132]:
process_tweet(test_pos[3]), test_pos[3]

(['stuck', 'centr', 'right', 'clown', 'right', 'joker', 'left', '...', ':)'],
 "@HumayAG 'Stuck in the centre right with you. Clowns to the right, jokers to the left...' :) @orgasticpotency @ahmedshaheed @AhmedSaeedGahaa")

In [117]:
def count_tweets(result, tweets, ys):
    for y, tweet in zip(ys, tweets):
        for word in process_tweet(tweet):
            key = (word, y)
            if key in result:
                result[key] += 1
            else:
                result[key] = 1
    return result


# count_tweets({}, train_pos[:10], [1]*10)

In [118]:
tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
labels = [1, 0, 0, 0, 0]
count_tweets({}, tweets, labels)

{('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 2}

## Trianing Using Naive Bayes
### How to train Naive Bayes Classifier
* first identify number of classes(categories/labels) that you have.
* probability for each class. calculate probability that the Document(tweet) is positive, and negative.
  $$ P\left( {D_{\text{pos}}} \right) = \frac{D_{\text{pos}}}{D} $$
  $$ P\left( {D_{\text{neg}}} \right) = \frac{D_{\text{pos}}}{D} $$

### Prior and Logprior:
The prior probability represents the underlying probability in the target population that a tweet is positive versus negative. In other words, if we have to pick a tweet out of all the tweet population, what is probability that it will be positive versus that it will be negative.

So, prior = $ \frac{P(D_{\text{pos}})}{P(D_{\text{pos}})} = \frac{D_{\text{pos}}}{D_{\text{neg}}} $ 

logprior = $ \log\left( {P(D_{\text{pos}})} \right) - \log\left( P(D_{\text{pos}}) \right) = \log\left( D_{\text{pos}} \right) - \log\left( D_{\text{neg}} \right) $

### Positive and Negative Probability of a Word
To compute the positive probability and the negative probability for a specific word in the vocabulary, we'll use the following inputs:
* $ freq_\text{pos} $ and $ freq_\text{neg} $  are the frequencies of that specific word in the positive or negative class. In other words, the positive frequency of a word is the number of times the word is counted with the label of 1.
* $ N_\text{pos} $ and $ N_\text{pos} $ are the total number of positive and negative words for all documents (for all tweets), respectively.
* $ V $ is the number of unique words in the entire set of documents, for all classes, whether positive or negative.
We'll use these to compute the positive and negative probability for a specific word using this formula:
 $$ P(W_\text{pos}) = \frac{freq_\text{pos} + 1}{N_\text{pos} + V}$$
 $$ P(W_\text{neg}) = \frac{freq_\text{neg} + 1}{N_\text{neg} + V}$$
+1 is added for smoothing(Laplace Smoothing).
##### Why Add-One Smoothing?
Without smoothing, if a word has  $ freq_\text{pos} = 0$ in positive documents, the probability $ P(W_\text{pos}) $ becomes 0. Since Naive Bayes multiplies probabilities across words, a zero probability for any word would make the entire document's probability for that class zero, even if other words strongly support the class.

Adding $ +1 $ensures every word has at least a small non-zero probability.
##### What does V do?
$ V $ (vocabulary size) is added to the denominator to account for the fact that we added $ +1 $ to the numerator for every word. It adjusts the probabilities to ensure they remain valid (sum to 1 across the entire vocabulary). Without $ V $, the probabilities could become inflated.

## Log likelihood
To compute the loglikelihood of that very same word, we can implement the following equations:
$$ loglikelihood = \log\left( {\frac{P(W_\text{pos})}{P(W_\text{neg})}}  \right) $$


In [120]:
freqs = count_tweets({}, train_x, train_y)
# freqs

### Instructions
1. calculate V (number of unique words appeared in the freqs dictionary)
2. calculate $ freq_\text{pos} $ and $ freq_\text{neg} $.
3. calculate $ N_\text{pos} $ and $ N_\text{neg} $
4. calculate $ D $, $ D_\text{pos} $ and $ D_\text{neg} $
5. calculate logprior
6. calculate log likelihood

In [121]:
def train_naive_bayes(freqs, train_x, train_y):
    loglikelihood = {}
    logprior = 0

    vocab = set([item[0] for item in freqs.keys()])
    V = len(vocab)
    # Npos, Nneg
    N_pos = N_neg = 0
    for key, count in freqs.items():
        if key[1] == 1:
            N_pos += count
        else:
            N_neg += 1

    # number of Documents(tweets)
    D = len(train_x)
    # number of positive Docs
    D_pos = len(train_pos)
    # number of negative Docs
    D_neg = len(train_neg)

    # logprior
    logprior = np.log(D_pos/D_neg)
    # i = 0
    for word in vocab:
        pos_freq = freqs.get((word, 1), 0)
        neg_freq = freqs.get((word, 0), 0)
        p_w_pos = (pos_freq + 1) / (N_pos + V)
        p_w_neg = (neg_freq + 1) / (N_neg + V)
        loglikelihood[word] = np.log(p_w_pos/p_w_neg)
        # if i == 10:
        #     break
        # i+= 1

    return logprior, loglikelihood
    




In [122]:
freqs[('happi', 1)], freqs[('happi', 0)]

(161, 18)

In [123]:
freqs[('sad', 1)], freqs[('sad', 0)]

(5, 100)

In [124]:
logprior, loglikelihood  = train_naive_bayes(freqs, train_x, train_y)
len(loglikelihood), logprior

(9085, np.float64(0.0))

## Part 3: Test your naive bayes
Now that we have the logprior and loglikelihood, we can test the naive bayes function by making predicting on some tweets!

**Implement** naive_bayes_predict
**Instructions:** Implement the naive_bayes_predict function to make predictions on tweets.
* The function takes in the tweet, logprior, loglikelihood.
* It returns the probability that the tweet belongs to the positive or negative class.
* For each tweet, sum up loglikelihoods of each word in the tweet.
* Also add the logprior to this sum to get the predicted sentiment of that tweet.
$$ p = logprior + \sum_{i}^{N} \left( loglikelihood_i \right)$$

##### Note
Note we calculate the prior from the training data, and that the training data is evenly split between positive and negative labels (4000 positive and 4000 negative tweets). This means that the ratio of positive to negative 1, and the logprior is 0.

The value of 0.0 means that when we add the logprior to the log likelihood, we're just adding zero to the log likelihood. However, please remember to include the logprior, because whenever the data is not perfectly balanced, the logprior will be a non-zero value.

In [125]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
    '''
    Input:
        tweet: a string
        logprior: a number
        loglikelihood: a dictionary of words mapping to numbers
    Output:
        p: the sum of all the logliklihoods of each word in the tweet (if found in the dictionary) + logprior (a number)
    '''
    words = process_tweet(tweet)
    p = 0
    p += logprior
    for word in words:
        p += loglikelihood.get(word, 0)
    return p

In [126]:
sample_tweets = ['She smiled.', 'i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
for tweet in sample_tweets:
    p = naive_bayes_predict(tweet, logprior, loglikelihood)
    print(f"The expected outpt is: {p}")

The expected outpt is: 0.6742938061462189
The expected outpt is: 1.2488352442983173
The expected outpt is: -0.8943221117676263
The expected outpt is: -3.717683159380831
The expected outpt is: -2.7401488022659573
The expected outpt is: -2.7401488022659573


In [127]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    """
    Input:
        test_x: A list of tweets
        test_y: The corresponding labels for the list of tweets
        logprior: The logprior
        loglikelihood: A dictionary with the loglikelihoods for each word

    Output:
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    accuracy = 0  # This will store the final accuracy

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    y_hats = []  # This will store the predicted labels

    for tweet, true_label in zip(test_x, test_y):
        # if the prediction is > 0
        if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
            # the predicted class is 1
            y_hat_i = 1
        else:
            # otherwise the predicted class is 0
            y_hat_i = 0

        # append the predicted class to the list y_hats
        y_hats.append(y_hat_i)

    # error is the average of the absolute values of the differences between y_hats and test_y
    correct_predictions = sum([1 for y_hat, y in zip(y_hats, test_y) if y_hat == y])
    error = (len(test_y) - correct_predictions) / len(test_y)
    print(f"{error=}")
    # Accuracy is 1 minus the error
    accuracy = 1 - error
    ### END CODE HERE ###
    return accuracy


In [128]:
print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

error=0.1195
Naive Bayes accuracy = 0.8805
